<a href="https://colab.research.google.com/github/forest1102/-/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Requirements:

In [11]:
!pip install tensorflow-gpu==2.0.0b1 cupy numpy

     |████████████████████████████████| 348.9MB 42kB/s 
     |████████████████████████████████| 3.7MB 30.5MB/s 
     |████████████████████████████████| 501kB 37.7MB/s 
     |████████████████████████████████| 3.1MB 31.0MB/s 
  Created wheel for cupy: filename=cupy-7.0.0-cp36-cp36m-linux_x86_64.whl size=28607969 sha256=f5a5709f422ad45d7d56c7777fff228c51dc4f910bd4633b4690d43faf49fbac
  Stored in directory: /root/.cache/pip/wheels/4b/33/37/f412224f7550a11ee18a9f41f7aff28df380bfa994f7280ab3
Successfully built cupy


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(-67.05016, shape=(), dtype=float32)


In [0]:
!mkdir dataset
!wget https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.train.txt -P /content/dataset
!wget https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.test.txt -P /content/dataset

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding, Lambda, Input, Reshape, Dot, Flatten
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard

import pickle

from util import create_contexts_target, most_similar
from dataset import ptb
from negative_sampling import generate_with_negative_sample

tensorboard_callback = TensorBoard(log_dir='logs/cbow')

window_size = 10
hidden_size = 100
batch_size = 100
max_epoch = 15
sample_size = 5

corpus, word_to_id, id_to_word = ptb.load_data('train')
test_corpus = ptb.load_data('test')[0]

vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
test_contexts, test_target = create_contexts_target(test_corpus, window_size)

print('corpus', corpus.shape)
print('contexts', contexts.shape)
print('target', target.shape)


contexts_input = Input(shape=(window_size * 2,), name='contexts_input')
target_input = Input(shape=(1,), name='target_input')

embed = Embedding(vocab_size, hidden_size, input_length=window_size * 2)

contexts_embed = embed(contexts_input)
contexts_hidden = Lambda(lambda arr: K.mean(arr, axis=1))(contexts_embed)

target_embed = Embedding(vocab_size, hidden_size, input_length=1)(target_input)
target_hidden = Reshape((hidden_size, ))(target_embed)

embed_dot = Dot(axes=1)([contexts_hidden, target_hidden])
output = Dense(1, activation='sigmoid')(embed_dot)

model = Model(inputs=[contexts_input, target_input], outputs=output)
print('corpus', corpus.shape)
print('contexts', contexts.shape)
print('target', target.shape)

model.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

print(model.summary())
hist = model.fit_generator(
    generate_with_negative_sample(
        corpus, contexts, target, batch_size, sample_size=sample_size),
    steps_per_epoch=len(contexts) // batch_size,
    initial_epoch=0,
    epochs=max_epoch, callbacks=[tensorboard_callback],
)

word_vecs = model.get_weights()[0]
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)



ModuleNotFoundError: ignored